# _Moneyballing Fantasy Premier League Football_
# Creating Tableau Data

One of the key outputs of this project is the production of Tableau dashboards.

This requires the creation of datasets, which are formatted in a way that is suitable for connection with Tableau.

### Bringing in the required libraries...

In [1]:
#Standard data manipulations
import pandas as pd
import numpy as np

#SQL
import sqlite3
#Including custom functions, stored elsewhere in the repo
from PythonFunctions.sqlfunctions import *

#Suppress warnings from showing
import warnings
warnings.filterwarnings('ignore')

We'll take out data from the SQL database that we've built over the course of the project.

In [2]:
#Connect to the database 'fpl.db' (fantasy premier league!)
conn = sqlite3.connect('Data/fpl.db')
#Instantiate a cursor
c = conn.cursor()

Making sure that this connection has worked as desired...

In [3]:
#Print the names of the tables in the SQL database
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res:
    print(name[0])

TeamsBasic
PlayersBasic
ShotsDetail
PlayerMatchesDetail
TeamMatchesDetail
FixturesBasic
TeamsXGC
PlayersXG
PlayerAPIStats
TeamsXG


We now bring in the various tables that we need, making slight edits to them as required.

In [4]:
df_pm = sql('SELECT * FROM PlayerMatchesDetail', c)
df_pm.drop(['TableIndex','RelativeStrength','ShotsCentre', 'ShotsLeft',
            'ShotsRight', 'CreatedCentre', 'CreatedLeft',
            'CreatedRight'], axis=1, inplace=True)

df_fixtures = sql('SELECT * FROM FixturesBasic', c).drop('TableIndex', axis=1)

df_player_basic = sql('SELECT * FROM PlayersBasic', c)

df_api_stats = sql('SELECT * FROM PlayerAPIStats', c)

df_xg = sql('SELECT * FROM PlayersXG', c)
df_xg['MatchID'] = df_xg['MatchID'].astype('str')

We now define some functions that will help us with table joins later on...

In [6]:
def pos_lookup(x):
    return df_player_basic.loc[df_player_basic['CommentName']==x,
                               'Position'].item()


def player_lookup(x):
    return df_player_basic.loc[df_player_basic['PlayerID']==x,
                               'CommentName'].item()

def team_lookup(x):
    return df_player_basic.loc[df_player_basic['PlayerID']==x,
                               'Team'].item()


def matchid_lookup(player, date, team, gameweek):
    try:
        try:
            return df_pm.loc[(df_pm['Date']==date)
                        &(df_pm['Player']==player), 'MatchID'].item()
        except:
            return df_pm.loc[(df_pm['Date']==date)
                        &(df_pm['ForTeam']==team), 'MatchID'].iloc[0]
    except:
        return df_pm.loc[(df_pm['GameWeek']==gameweek)
                        &(df_pm['Player']==player), 'MatchID'].item()
    
    
def api_stat_lookup(player, match_id):
    
    columns=['MinutesPlayed', 'Points', 'Price', 'SelectedBy','NetTransfersIn']

    return df_api_stats.loc[(df_api_stats['Player']==player)
                &(df_api_stats['MatchID']==match_id), columns]    

We now mung our data together...

In [7]:
df_home = df_fixtures.drop('AwayTeam', axis=1)
df_away = df_fixtures.drop('HomeTeam', axis=1)

df_home['Home']=df_home['MatchID'].map(lambda x: 'Home')
df_away['Home']=df_away['MatchID'].map(lambda x: 'Away')

df_away

cols = ['MatchID','GameWeek','Date','Team','Home']
df_home.columns = cols
df_away.columns = cols

#Combine the above dataframes, sorting by matchid
df_melt = pd.concat([df_home,df_away])
df_melt.sort_values(['MatchID','Home'], ascending=[True, False], inplace=True)

AgainstTeam = []
#Get the opposing team
for row in df_melt.itertuples():
    #For each row get the match ID
    match = row.MatchID
    #And the required team
    home = row.Home
    if home == 'Away':
        home = 'Home'
    else:
        home = 'Away'
    home = f'{home}Team'
    requiredTeam = df_fixtures.loc[(df_fixtures['MatchID']==match),home].item()
    AgainstTeam.append(requiredTeam)
    
df_melt['AgainstTeam'] = AgainstTeam
df_melt.reset_index(inplace=True, drop=True)


#Instantiate an empty list
match_ids = []
home_away = []
dates = []

#For each row in the player matches dataframe...
for row in df_pm.itertuples():
    #Look up the match id from the team matches dataframe
    team = row.ForTeam
    againstteam = row.AgainstTeam
    gameweek = row.GameWeek

    match_id = df_melt.loc[(df_melt['GameWeek']==gameweek)
                          &(df_melt['Team']==team)
                          &(df_melt['AgainstTeam']==againstteam),
                          'MatchID'].item()
    
    date = df_melt.loc[(df_melt['GameWeek']==gameweek)
                          &(df_melt['Team']==team)
                          &(df_melt['AgainstTeam']==againstteam),
                          'Date'].item()
    
    home = df_melt.loc[(df_melt['GameWeek']==gameweek)
                          &(df_melt['Team']==team)
                          &(df_melt['AgainstTeam']==againstteam),
                          'Home'].item()
    
    #Add it to the list
    match_ids.append(match_id)
    home_away.append(home)
    dates.append(date)
    
#Declare the list as a column in the player matches df
df_pm['MatchID']=match_ids
df_pm['Date']=pd.to_datetime(dates)
df_pm['Home']=home_away
df_pm['Position']=df_pm['Player'].map(pos_lookup)

#Get the match IDs column first in the dataframe
cols = list(df_pm.columns)
new_cols = ['MatchID', 'Date', 'Home','Position'] + cols[:-4]
df_pm = df_pm[new_cols]

#Bring in stats from api table
#First, get key identifiers into the api table to facilitate joining
df_api_stats['Player'] = df_api_stats['PlayerID'].map(player_lookup)
df_api_stats['Team'] = df_api_stats['PlayerID'].map(team_lookup)
df_api_stats['MatchID'] = df_api_stats.apply(lambda x: matchid_lookup(x['Player'],
                                                                      x['Date'],
                                                                      x['Team'],
                                                                      x['GameWeek']), axis=1)


api_cols = ['Player', 'MatchID', 'BPS', 'MinutesPlayed',
            'CleanSheet', 'Saves', 'NetTransfersIn',
            'SelectedBy', 'Points', 'Price']

df_api_cols = df_api_stats[api_cols]

And create a new dataframe with all the required data, formatted as required.

In [9]:
df_test=pd.merge(df_pm, df_api_cols,
         how='left',
         left_on=['Player','MatchID'],
         right_on = ['Player','MatchID'])

df_test.drop('Minutes', inplace=True, axis=1)

int_cols = ['BPS', 'MinutesPlayed', 'CleanSheet',
            'Saves', 'NetTransfersIn',
            'SelectedBy', 'Points']

df_test.sort_values(['Date','ForTeam','Position','Player'], inplace=True)
df_test.reset_index(drop=True, inplace=True)

df_test.columns = ['MatchID', 'Date', 'Home or Away', 'Position', 'Player', 'GameWeek', 'For Team',
       'Against Team', 'Goals', 'Shots On Target', 'Shots In The Box', 'Close Shots',
       'Total Shots', 'Headers', 'Assists', 'Shots On Target Created',
       'Shots In Box Created', 'Close Shots Created', 'Total Shots Created',
       'Headers Created', 'Bonus Points System', 'Minutes Played', 'Clean Sheet', 'Saves',
       'Net Transfers In', 'Managers Selected By', 'Points', 'Price']

df_test=pd.merge(df_test, df_xg[['Player', 'MatchID', 'XG', 'XA', 'XGI']],
         how='left',
         left_on=['Player','MatchID'],
         right_on = ['Player','MatchID'])

df_test.drop_duplicates(subset=['MatchID', 'Player', 'For Team', 'Against Team'], inplace=True)

df_test.head()

Save the whole lot down into a CSV file.

In [12]:
df_test.to_csv('Data/tableau_p_data.csv')

## Now team data...

In [13]:
df_tm = sql('SELECT * FROM TeamMatchesDetail', c)

def date_lookup(x):
    x=str(x)
    return df_fixtures.loc[df_fixtures['MatchID']==x, 'Date'].item()

df_tm['Date'] = df_tm['MatchID'].map(date_lookup)

df_txg = sql('SELECT * FROM TeamsXG', c)

df_tm = pd.merge(df_tm, df_txg[['ForTeam', 'MatchID', 'XG', 'XGC']],
                 how='left',
                 left_on=['ForTeam','MatchID'],
                 right_on = ['ForTeam','MatchID'])

df_tm = df_tm[['MatchID', 'ForTeam', 'AgainstTeam', 'Date',
               'GameWeek', 'Home', 'Possession', 'Touches', 'Passes',
               'Tackles', 'Clearances', 'Corners', 'Offsides', 'FoulsConceded',
               'YellowCards', 'Goals', 'XG', 'ShotsOnTarget', 'TotalShots',
               'GoalsConceded', 'ShotsConceded', 'XGC','ShotsInBox', 'CloseShots',
               'Headers', 'ShotsCentre', 'ShotsLeft', 'ShotsRight', 'ShotsInBoxConceded',
               'CloseShotsConceded', 'HeadersConceded', 'ShotsCentreConceded',
               'ShotsLeftConceded', 'ShotsRightConceded']]

df_tm.columns = ['MatchID', 'For Team', 'Against Team', 'Date',
               'GameWeek', 'Home', 'Possession', 'Touches', 'Passes',
               'Tackles', 'Clearances', 'Corners', 'Offsides', 'Fouls Committed',
               'Yellow Cards', 'Goals', 'XG', 'Shots On Target', 'Total Shots',
               'Goals Conceded', 'Shots Conceded', 'XGC','Shots In Box', 'Close Shots',
               'Headers', 'Shots Centre', 'Shots Left', 'Shots Right', 'Shots In Box Conceded',
               'Close Shots Conceded', 'Headers Conceded', 'Shots Centre Conceded',
               'Shots Left Conceded', 'Shots Right Conceded']

df_tm.drop_duplicates(subset=['MatchID', 'For Team', 'Against Team'], inplace=True)

df_tm.head()

,MatchID,For Team,Against Team,Date,GameWeek,Home,Possession,Touches,Passes,Tackles,...,Headers,Shots Centre,Shots Left,Shots Right,Shots In Box Conceded,Close Shots Conceded,Headers Conceded,Shots Centre Conceded,Shots Left Conceded,Shots Right Conceded
0,46605,Liverpool,Norwich City,2019-08-09,1,Home,57.9,662,477,21,...,2,6,2,2,7,7,1,4,3,1
1,46605,Norwich City,Liverpool,2019-08-09,1,Away,42.1,577,363,12,...,1,4,3,1,10,9,2,6,2,2
2,46606,Bournemouth,Sheffield United,2019-08-10,1,Home,52.9,671,485,18,...,1,4,1,1,2,4,0,2,0,0
3,46606,Sheffield United,Bournemouth,2019-08-10,1,Away,47.1,615,428,14,...,0,2,0,0,6,4,1,4,1,1
4,46607,Burnley,Southampton,2019-08-10,1,Home,46.5,549,332,21,...,2,3,1,4,5,3,0,1,4,0


In [14]:
df_tm.to_csv('Data/tableau_t_data.csv')